In [ ]:
import rioxarray as rio
import xarray as xr
import glob
import os
import numpy as np
import requests
import geopandas as gpd
from pathlib import Path
from datetime import datetime
from rasterio.enums import Resampling
%matplotlib inline

all_scenes_f_precip = Path('/scratch/waves/rhone-ecostress/rasters/chirps-clipped')
all_scenes_f_et = Path('/home/serdp/rhone/rhone-ecostress/rasters/eeflux/PDR')

all_precip_paths = list(all_scenes_f_precip.glob("*"))
all_pdr_et_paths = list(all_scenes_f_et.glob("*.tif"))

In [ ]:
# for some reason the fll value is not correct. this is the correct bad value to mask by
testf = all_precip_paths[0]
x = rio.open_rasterio(testf)
badvalue = np.unique(x.where(x != x._FillValue).sel(band=1))[0]

def chirps_path_date(path):
    _, _, year, month, day, _ = path.name.split(".")
    day = day.split("-")[0]
    return datetime(int(year), int(month), int(day))


def open_chirps(path):
    data_array = rio.open_rasterio(path) #chunks makes i lazyily executed
    data_array = data_array.sel(band=1).drop("band") # gets rid of old coordinate dimension since we need bands to have unique coord ids
    data_array["date"] = chirps_path_date(path) # makes a new coordinate
    return data_array.expand_dims({"date":1}) # makes this coordinate a dimension



### data is not tiled so not a good idea to use chunking
#https://github.com/pydata/xarray/issues/2314

import rasterio
with rasterio.open(testf) as src:
    print(src.profile)

len(all_precip_paths) * 41.7 / 10e3

%timeit open_chirps(testf)

all_daily_precip_path = "/home/serdp/ravery/rhone-ecostress/netcdfs/all_chirps_daily.nc"

if Path(all_daily_precip_path).exists():
    
    all_chirps_arr = xr.open_dataarray(all_daily_precip_path)
    all_chirps_arr = all_chirps_arr.sortby("date")
else:

    daily_chirps_arrs = []

    for path in all_precip_paths:

        daily_chirps_arrs.append(open_chirps(path)) 
        
    all_chirps_arr = xr.concat(daily_chirps_arrs, dim="date")
    
    all_chirps_arr = all_chirps_arr.sortby("date")

    all_chirps_arr.to_netcdf(all_daily_precip_path)

def eeflux_path_date(path):
    year, month, day, _ = path.name.split("_")
    return datetime(int(year), int(month), int(day))

def open_eeflux(path, da_for_match):
    data_array = rio.open_rasterio(path) #chunks makes i lazyily executed
    data_array.rio.reproject_match(da_for_match)
    data_array = data_array.sel(band=1).drop("band") # gets rid of old coordinate dimension since we need bands to have unique coord ids
    data_array["date"] = eeflux_path_date(path) # makes a new coordinate
    return data_array.expand_dims({"date":1}) # makes this coordinate a dimension

da_for_match = rio.open_rasterio(all_pdr_et_paths[0])
daily_eeflux_arrs = [open_eeflux(path, da_for_match) for path in all_pdr_et_paths]
all_eeflux_arr = xr.concat(daily_eeflux_arrs, dim="date")
all_daily_eeflux_path = "/home/serdp/ravery/rhone-ecostress/netcdfs/all_eeflux_daily.nc"
all_eeflux_arr.to_netcdf(all_daily_eeflux_path)

all_eeflux_arr[-3,:,:].plot.imshow()

all_eeflux_arr = all_eeflux_arr.sortby("date")

In [ ]:
def years_list(all_eeflux_arr):
    ey = max(all_eeflux_arr['date.year'].values)
    sy = min(all_eeflux_arr['date.year'].values)
    start_years = range(sy,ey)
    end_years = range(sy+1, ey+1)
    return list(zip(start_year,end_year))

def group_by_custom_doy(all_eeflux_arr, doy_start, doy_end):
    start_end_years = years_list(all_eeflux_arr)
    water_year_arrs = []
    for water_year in start_end_years:
        start_mask = ((all_eeflux_arr['date.dayofyear'].values > doy_start) & (all_eeflux_arr['date.year'].values == water_year[0]))
        end_mask = ((all_eeflux_arr['date.dayofyear'].values < doy_end) & (all_eeflux_arr['date.year'].values == water_year[1]))
        water_year_arrs.append(all_eeflux_arr[start_mask | end_mask])
    return water_year_arrs
doystart = 275
doyend = 125
water_year_arrs = group_by_custom_doy(all_eeflux_arr, doystart, doyend)

In [ ]:
group_counts = list(map(lambda x: len(x['date']), water_year_arrs))
year_tuples = years_list(all_eeflux_arr)

indexes = np.arange(len(year_tuples))
plt.bar(indexes, group_counts)
degrees = 80
plt.xticks(indexes, year_tuples, rotation=degrees, ha="center")
plt.title("Availability of EEFLUX between DOY 275 and 125")
plt.savefig("eeflux_availability.png")

In [ ]:
def sum_seasonal_precip(precip_arr, eeflux_group_arr):
    return precip_arr.sel(date=slice(eeflux_group_arr.date.min(), eeflux_group_arr.date.max())).sum(dim="date")

In [ ]:
for index, eeflux_group in enumerate(water_year_arrs):
    if len(eeflux_group['date']) > 0:
        seasonal_precip = sum_seasonal_precip(all_chirps_arr, eeflux_group)
        seasonal_et = eeflux_group.integrate(dim="date", datetime_unit="D")
        year_range = year_tuples[index]
        pname = f"seasonal_chirps_{year_range[0]}_{year_range[1]}_{doystart}_{doyend}.tif"
        eename = f"seasonal_eeflux_integrated_{year_range[0]}_{year_range[1]}_{doystart}_{doyend}.tif"
        seasonal_precip.rio.to_raster(pname)
        seasonal_et.rio.to_raster(eename)

In [ ]:
seasonal_precip

In [ ]:
water_year_arrs[0][0].plot.imshow()

In [ ]:
water_year_arrs[0].integrate(dim="date", datetime_unit="D").plot.imshow()

In [ ]:
all_eeflux_arr.groupby(????????).integrate(dim="date", datetime_unit="D")

In [ ]:
import pandas as pd
import numpy as np

labels = ['<=2', '3-9', '>=10']
bins = [0,2,9, np.inf]
pd.cut(all_eeflux_arr, bins, labels=labels)

In [ ]:
all_eeflux_arr

In [ ]:
import pandas as pd

all_scene_ids = [str(i) for i in list(all_scenes_f.glob("L*"))]
df = pd.DataFrame({"scene_id":all_scene_ids}).reindex()
split_vals_series = df.scene_id.str.split("/")

dff = pd.DataFrame(split_vals_series.to_list(), columns=['_', '__', '___', '____', '_____', '______', 'fname'])

df['date'] = dff['fname'].str.slice(10,18)

df['pathrow'] = dff['fname'].str.slice(4,10)

df['sensor'] = dff['fname'].str.slice(0,4)

df['datetime'] = pd.to_datetime(df['date'])

df = df.set_index("datetime").sort_index()

In [ ]:
marc_df = df['2014-01-01':'2019-12-31']

In [ ]:
marc_df = marc_df[marc_df['sensor']=="LC08"]

In [ ]:

x.where(x != badvalue).sel(band=1).plot.imshow()